In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer

In [2]:
df = pd.read_excel('./data/aug_nine_var.xlsx').iloc[:,1::]
df.columns = ["합계 출산율", "국내 총인구", "서울시 인구", "경기도 인구", "인천시 인구", "수도권 인구", "신생아 출산 인구", "사망 인구", "혼인 건수", "이혼 건수", "소비자 물가", "농축 수산물", "공업 제품", "집세", "공공 서비스", "개인 서비스", "소비자 물가 지수", "금리", "원 달러 환율", "국제 유가", "국내 총생산", "수도권 과밀화율"]

In [3]:
df.sort_values("수도권 과밀화율")

,합계 출산율,국내 총인구,서울시 인구,경기도 인구,인천시 인구,수도권 인구,신생아 출산 인구,사망 인구,혼인 건수,이혼 건수,...,공업 제품,집세,공공 서비스,개인 서비스,소비자 물가 지수,금리,원 달러 환율,국제 유가,국내 총생산,수도권 과밀화율
8,1.580,41265113,9798542,5075449,1441131,16315122,50185,17666,19443,3174,...,2.0,3.8,4.4,5.4,30.619,9.7,879.11,14.75,102985.8,36.092
7,1.580,41265113,9798542,5075449,1441131,16315122,49608,17963,22861,3473,...,2.1,3.8,4.4,5.4,30.442,9.7,882.37,15.90,102985.8,36.100
9,1.580,41265113,9798542,5075449,1441131,16315122,52638,20247,24908,3153,...,2.0,3.6,4.2,5.1,30.323,9.7,874.63,15.28,102985.8,36.105
6,1.580,41265113,9798542,5075449,1441131,16315122,48895,18167,27723,3332,...,2.0,3.8,4.4,5.6,30.471,9.7,885.04,11.15,102985.8,36.112
5,1.580,41265113,9798542,5075449,1441131,16315122,45245,17587,31120,2944,...,2.1,4.0,4.2,5.6,30.442,9.7,888.34,13.85,102985.8,36.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,1.259,49034810,10421782,11340241,2664576,24426599,44871,19857,18284,8673,...,0.2,0.1,0.7,-0.1,78.655,5.0,932.41,81.66,1089660.2,44.380
261,1.259,49034810,10421782,11340241,2664576,24426599,45722,21531,27323,11329,...,0.4,0.1,0.0,0.1,78.803,5.0,915.86,94.53,1089660.2,44.380
263,1.259,49034810,10421782,11340241,2664576,24426599,36719,21822,39767,9709,...,0.7,0.2,0.1,0.3,79.101,5.0,930.24,96.00,1089660.2,44.380
262,1.259,49034810,10421782,11340241,2664576,24426599,41865,21170,33472,10949,...,1.0,0.3,0.4,0.0,78.803,5.0,916.98,88.71,1089660.2,44.380


In [4]:
hundred_percent = df
ninety_percent = df.iloc[0:355]
eighty_percent = df.iloc[0:316]
seventy_percent = df.iloc[0:276]
sixty_percent = df.iloc[0:237]

In [5]:
ninety_percent_X = ninety_percent.iloc[:, 0:21]
ninety_percent_y = ninety_percent.iloc[:, 21:22].to_numpy()
eighty_percent_X = eighty_percent.iloc[:, 0:21]
eighty_percent_y = eighty_percent.iloc[:, 21:22].to_numpy()
seventy_percent_X = seventy_percent.iloc[:, 0:21]
seventy_percent_y = seventy_percent.iloc[:, 21:22].to_numpy()
sixty_percent_X = sixty_percent.iloc[:, 0:21]
sixty_percent_y = sixty_percent.iloc[:, 21:22].to_numpy()

In [6]:
scaler = MinMaxScaler()
X = scaler.fit_transform(ninety_percent_X)
y = ninety_percent_y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

In [7]:
input_layer = tf.keras.layers.Input(shape=(21,))

encoded = tf.keras.layers.Dense(128, activation='relu')(input_layer)
encoded = tf.keras.layers.Dense(64, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(32, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(16, activation='relu')(encoded)
encoded = tf.keras.layers.Dense(3)(encoded)
 
decoded = tf.keras.layers.Dense(3, activation='relu')(encoded)
decoded = tf.keras.layers.Dense(16, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(32, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(64, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(128, activation='relu')(decoded)
decoded = tf.keras.layers.Dense(21)(decoded)
 
# autoencoder
autoencoder = tf.keras.Model(input_layer, decoded)
 
# encoder
encoder = tf.keras.Model(input_layer, encoded)

# decoder
encoded_input = tf.keras.layers.Input(shape=(3,))

decoded_layer = autoencoder.layers[6](encoded_input)
decoded_layer = autoencoder.layers[7](decoded_layer)
decoded_layer = autoencoder.layers[8](decoded_layer)
decoded_layer = autoencoder.layers[9](decoded_layer)
decoded_layer = autoencoder.layers[10](decoded_layer)
decoded_layer = autoencoder.layers[11](decoded_layer)

decoder = tf.keras.Model(encoded_input, decoded_layer)
 
ninety_percent_ae = autoencoder
eighty_percent_ae = autoencoder
seventy_percent_ae = autoencoder
sixty_percent_ae = autoencoder

ninety_percent_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])
eighty_percent_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])
seventy_percent_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])
sixty_percent_ae.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [8]:
X = scaler.fit_transform(ninety_percent_X)
y = ninety_percent_y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

ninety_hist = ninety_percent_ae.fit(X_train, X_train,
                batch_size=32, epochs=1000,
                validation_data=(X_test,X_test),
                verbose=1)

ninety_percent_encoder_out = encoder.predict(X)
enc_out = pd.DataFrame(ninety_percent_encoder_out)
enc_out.to_excel('ninety_enc_out.xlsx')

Epoch 1/1000
9/9 [==============================] - 1s 26ms/step - loss: 0.2432 - mae: 0.4164 - val_loss: 0.2151 - val_mae: 0.3760
Epoch 2/1000
9/9 [==============================] - 0s 5ms/step - loss: 0.1667 - mae: 0.3283 - val_loss: 0.1075 - val_mae: 0.2615
Epoch 3/1000
9/9 [==============================] - 0s 5ms/step - loss: 0.0864 - mae: 0.2425 - val_loss: 0.0711 - val_mae: 0.2274
Epoch 4/1000
9/9 [==============================] - 0s 5ms/step - loss: 0.0591 - mae: 0.2032 - val_loss: 0.0624 - val_mae: 0.2103
Epoch 5/1000
9/9 [==============================] - 0s 5ms/step - loss: 0.0555 - mae: 0.1964 - val_loss: 0.0614 - val_mae: 0.2081
Epoch 6/1000
9/9 [==============================] - 0s 5ms/step - loss: 0.0535 - mae: 0.1924 - val_loss: 0.0591 - val_mae: 0.2048
Epoch 7/1000
9/9 [==============================] - 0s 4ms/step - loss: 0.0522 - mae: 0.1894 - val_loss: 0.0579 - val_mae: 0.2016
Epoch 8/1000
9/9 [==============================] - 0s 6ms/step - loss: 0.0512 - mae: 0.1

In [ ]:
X = scaler.fit_transform(eighty_percent_X)
y = eighty_percent_y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

eighty_hist = eighty_percent_ae.fit(X_train, X_train,
                batch_size=32, epochs=1000,
                validation_data=(X_test,X_test),
                verbose=1)

encoder_out = encoder.predict(X)
enc_out = pd.DataFrame(encoder_out)
enc_out.to_excel('eighty_enc_out.xlsx')

Epoch 1/1000
8/8 [==============================] - 0s 9ms/step - loss: 0.0041 - mae: 0.0432 - val_loss: 0.0051 - val_mae: 0.0460
Epoch 2/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0039 - mae: 0.0413 - val_loss: 0.0049 - val_mae: 0.0440
Epoch 3/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0034 - mae: 0.0387 - val_loss: 0.0046 - val_mae: 0.0411
Epoch 4/1000
8/8 [==============================] - 0s 6ms/step - loss: 0.0034 - mae: 0.0373 - val_loss: 0.0045 - val_mae: 0.0407
Epoch 5/1000
8/8 [==============================] - 0s 5ms/step - loss: 0.0034 - mae: 0.0380 - val_loss: 0.0045 - val_mae: 0.0419
Epoch 6/1000
8/8 [==============================] - 0s 6ms/step - loss: 0.0033 - mae: 0.0374 - val_loss: 0.0043 - val_mae: 0.0408
Epoch 7/1000
8/8 [==============================] - 0s 4ms/step - loss: 0.0033 - mae: 0.0377 - val_loss: 0.0045 - val_mae: 0.0401
Epoch 8/1000
8/8 [==============================] - 0s 6ms/step - loss: 0.0031 - mae: 0.03

In [ ]:
X = scaler.fit_transform(seventy_percent_X)
y = seventy_percent_y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

seventy_hist = seventy_percent_ae.fit(X_train, X_train,
                batch_size=32, epochs=1000,
                validation_data=(X_test,X_test),
                verbose=1)

encoder_out = encoder.predict(X)
enc_out = pd.DataFrame(encoder_out)
enc_out.to_excel('seventy_enc_out.xlsx')

Epoch 1/1000
7/7 [==============================] - 0s 18ms/step - loss: 0.0061 - mae: 0.0537 - val_loss: 0.0071 - val_mae: 0.0548
Epoch 2/1000
7/7 [==============================] - 0s 11ms/step - loss: 0.0045 - mae: 0.0477 - val_loss: 0.0054 - val_mae: 0.0470
Epoch 3/1000
7/7 [==============================] - 0s 12ms/step - loss: 0.0035 - mae: 0.0406 - val_loss: 0.0053 - val_mae: 0.0454
Epoch 4/1000
7/7 [==============================] - 0s 12ms/step - loss: 0.0033 - mae: 0.0382 - val_loss: 0.0050 - val_mae: 0.0413
Epoch 5/1000
7/7 [==============================] - 0s 14ms/step - loss: 0.0031 - mae: 0.0366 - val_loss: 0.0043 - val_mae: 0.0383
Epoch 6/1000
7/7 [==============================] - 0s 12ms/step - loss: 0.0027 - mae: 0.0347 - val_loss: 0.0046 - val_mae: 0.0394
Epoch 7/1000
7/7 [==============================] - 0s 12ms/step - loss: 0.0025 - mae: 0.0334 - val_loss: 0.0050 - val_mae: 0.0419
Epoch 8/1000
7/7 [==============================] - 0s 12ms/step - loss: 0.0024 - m

In [ ]:
X = scaler.fit_transform(sixty_percent_X)
y = sixty_percent_y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

sixty_hist = sixty_percent_ae.fit(X_train, X_train,
                batch_size=32, epochs=1000,
                validation_data=(X_test,X_test),
                verbose=1)

encoder_out = encoder.predict(X)
enc_out = pd.DataFrame(encoder_out)
enc_out.to_excel('sixty_enc_out.xlsx')

Epoch 1/1000
6/6 [==============================] - 0s 13ms/step - loss: 0.0078 - mae: 0.0578 - val_loss: 0.0073 - val_mae: 0.0585
Epoch 2/1000
6/6 [==============================] - 0s 5ms/step - loss: 0.0058 - mae: 0.0535 - val_loss: 0.0076 - val_mae: 0.0559
Epoch 3/1000
6/6 [==============================] - 0s 6ms/step - loss: 0.0054 - mae: 0.0506 - val_loss: 0.0065 - val_mae: 0.0547
Epoch 4/1000
6/6 [==============================] - 0s 7ms/step - loss: 0.0041 - mae: 0.0455 - val_loss: 0.0055 - val_mae: 0.0491
Epoch 5/1000
6/6 [==============================] - 0s 7ms/step - loss: 0.0038 - mae: 0.0425 - val_loss: 0.0053 - val_mae: 0.0484
Epoch 6/1000
6/6 [==============================] - 0s 7ms/step - loss: 0.0032 - mae: 0.0398 - val_loss: 0.0052 - val_mae: 0.0441
Epoch 7/1000
6/6 [==============================] - 0s 5ms/step - loss: 0.0030 - mae: 0.0370 - val_loss: 0.0043 - val_mae: 0.0404
Epoch 8/1000
6/6 [==============================] - 0s 6ms/step - loss: 0.0025 - mae: 0.0